In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("BostonHousing.csv")
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['medv'])
y = df['medv']
cols = X.columns

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X,columns=cols)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_test:',X_test.shape)
print('y_test:',y_test.shape)


X_train: (404, 13)
y_train: (404,)
X_test: (102, 13)
y_test: (102,)


In [27]:
class NeuralNetwork:
    def __init__(self, input_neurons, hidden_neurons, output_neurons, learning_rate):
        self.weights_input_hidden = np.random.uniform(-0.5, 0.5, (hidden_neurons, input_neurons))
        self.weights_hidden_output = np.random.uniform(-0.5, 0.5, (output_neurons, hidden_neurons))
        self.learning_rate = learning_rate
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def forward(self, inputs):
        self.inputs = np.array(inputs, ndmin=2).T
        # @ it is like np.dot
        hidden_input = self.weights_input_hidden @ self.inputs
        self.hidden_output = self.sigmoid(hidden_input)
        final_input = self.weights_hidden_output @ self.hidden_output
        return final_input
    
    def backward(self, actual, pred):
        output_error = actual - pred.T
        hidden_grad = (self.weights_hidden_output.T @ output_error) * (self.hidden_output) * (1 - self.hidden_output)
        self.weights_hidden_output += self.learning_rate * output_error * self.hidden_output.T
        self.weights_input_hidden += self.learning_rate * hidden_grad * self.inputs.T
    def MSE(self,actual,pred):
        return (np.sum(actual-pred)**2) / len(actual)        

In [28]:
nn = NeuralNetwork(X_train.shape[1],32,1,0.01)
epochs= 5000
losses = {'train':[],'test':[]}
for epoch in range(epochs):
    batch =np.random.choice(X_train.index,8, replace = False)
    for features,target in zip(X_train.loc[batch].values,y_train.loc[batch].values):
        pred = nn.forward(features)
        nn.backward(target, pred)
    losses['train'].append(nn.MSE(y_train,nn.forward(X_train)[0]))
    losses['test'].append(nn.MSE(y_test,nn.forward(X_test)[0]))


In [18]:
result = pd.DataFrame(
    {
        'Actual':y_test.values,
        'Prediction': nn.forward(X_test)[0]
    }
)
result

,Actual,Prediction
0,23.3,25.680411
1,21.7,21.673816
2,18.4,17.898813
3,12.1,13.289166
4,14.5,15.179079
...,...,...
97,36.0,34.179170
98,18.7,21.384871
99,19.1,15.865467
100,13.9,13.063168
